In [29]:
import numpy as np
import scipy.linalg as la
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import scipy.special as sp

In [33]:
def gram_schmidt_orthogonalization(n):
    X, W = np.polynomial.legendre.leggauss(n)
    V = np.vander(X, increasing=True)
    G = V.T @ np.diag(W) @ V
    Q = la.cholesky(G, lower=True)
    Q_inv = la.inv(Q)
    ortho_polys = [np.poly1d(Q_inv[i, ::-1]) for i in range(n)]
    return ortho_polys

def recurrence_relation(n):
    polys = [np.poly1d([1])]
    if n > 1:
        polys.append(np.poly1d([1, 0]))
    for k in range(2, n):
        Pk = ((2*k - 1) * np.poly1d([1, 0]) * polys[k-1] - (k - 1) * polys[k-2]) / k
        polys.append(Pk)
    return polys

def check_orthogonality(polys, n):
    X, W = np.polynomial.legendre.leggauss(n)
    V = np.array([[p(x) for p in polys] for x in X])
    G = V.T @ np.diag(W) @ V
    return np.allclose(G - np.diag(np.diag(G)), 0)

n = 10
gs_polys = gram_schmidt_orthogonalization(n)
rec_polys = recurrence_relation(n)

print("Gram-Schmidt:")
print("Orthogonality:", check_orthogonality(gs_polys, n))

print()

print("Recurrent relations:")
print("Orthogonality:", check_orthogonality(rec_polys, n))

Gram-Schmidt:
Orthogonality: True

Recurrent relations:
Orthogonality: True


In [34]:
import numpy as np
import scipy.linalg as la

def create_jacobi_matrix(n, alpha=1e-9, beta=1e-9):
    J = np.zeros((n, n))
    for i in range(n - 1):
        J[i, i + 1] = np.sqrt((i + 1) * (i + alpha) / (2 * (i + beta)))
        J[i + 1, i] = J[i, i + 1]
    for i in range(n):
        J[i, i] = (2 * i + alpha + beta)
    return J

def orthogonal_polynomials_from_jacobi_matrix(jacobi_matrix):
    eigenvalues, eigenvectors = la.eig(jacobi_matrix)
    eigenvalues = eigenvalues.real
    eigenvectors = eigenvectors.real
    norm_eigenvectors = np.zeros_like(eigenvectors)
    
    for i in range(eigenvectors.shape[1]):
        norm_eigenvectors[:, i] = eigenvectors[:, i] / np.linalg.norm(eigenvectors[:, i])

    polynomials = []
    for i in range(norm_eigenvectors.shape[1]):
        polynomials.append(norm_eigenvectors[:, i])
    
    return polynomials

def check_orthogonality(polynomials, epsilon=1e-10):
    n = len(polynomials)
    for i in range(n):
        for j in range(i + 1, n):
            dot_product = np.dot(polynomials[i], polynomials[j])
            if abs(dot_product) > epsilon:
                print(f"Polynomials {i+1} and {j+1} not orthogonal! Dot product: {dot_product}")
                return False
    return True

n = 10
jacobi_matrix = create_jacobi_matrix(n)
polynomials = orthogonal_polynomials_from_jacobi_matrix(jacobi_matrix)
print("Orthogonal:", check_orthogonality(polynomials))


Orthogonal: True
